# NMEA Reader - National Marine Electronics Association

Created by Michael George (AKA Logiqx)

This module was created because the NMEAReader of pynmeagps was way too slow for my liking.

NMEAReader took around 16 seconds for my test file, whereas populating numpy arrays with this module took about 2 seconds.

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import io
import time

from datetime import datetime, timezone

import numpy as np

import unittest

from file_reader import FileReader

## Main Class

In [2]:
class NmeaReader(FileReader):
    '''NMEA file - National Marine Electronics Association'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self, ignoreChecksums=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.readlines()
                       
            if len(self.buffer) > 0:
                self.readData(ignoreChecksums=ignoreChecksums)


    def readData(self, ignoreChecksums=False):
        '''Read GGA and RMC messages into memory'''

        # Lean buffers will only contain the actual payloads
        leanRmc = bytearray()
        leanGga = bytearray()

        numRmcFields = numGgaFields = 0

        for sentence in self.buffer:
            typeCode, payload = self.readSentence(sentence, ignoreChecksums=ignoreChecksums)

            if typeCode == b'RMC':
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanRmc += payload
                leanRmc += b'\n'

                if numRmcFields == 0:
                    numRmcFields = payload.count(b',') + 1

            elif typeCode == b'GGA':
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanGga += payload
                leanGga += b'\n'

                if numGgaFields == 0:
                    numGgaFields = payload.count(b',') + 1

        dtype = self.getRmcDatatype(numRmcFields)
        self.rmcData = np.genfromtxt(io.BytesIO(leanRmc), dtype=dtype, delimiter=",")

        dtype = self.getGgaDatatype(numGgaFields)
        self.ggaData = np.genfromtxt(io.BytesIO(leanGga), dtype=dtype, delimiter=",")

        self.consumeRawData()


    def readSentence(self, sentence, ignoreChecksums=False):
        '''Process NMEA sentence'''

        # Check for $ symbol (ASCII 0x24)
        if sentence[0] != 0x24:
            raise ValueError('Sentence does not start with a dollar $ symbol')
            
        asteriskIdx = sentence.find(b'*')
        if asteriskIdx == -1:
            raise ValueError('Sentence does not contain an asterisk * symbol')
        
        # Evaluate checksum
        if ignoreChecksums == False:
            expectedChecksum = int(sentence[asteriskIdx + 1: asteriskIdx + 3], 16)

            actualChecksum = 0
            for byte in sentence[1:asteriskIdx]:
                actualChecksum ^= byte

            if actualChecksum != expectedChecksum:
                raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        typeCode = sentence[3:6]
        payload = sentence[7:asteriskIdx]

        return typeCode, payload


    def getRmcDatatype(self, numFields):
        '''Get the datatype of the RMC payload'''

        # Standard RMC payload
        dtype = [
            ('hhmmss', 'f4'),     # UTC of this position report as hhmmss.ss
            ('status', 'U1'),     # Status, A = Valid, V = Warning
            ('lat', 'f8'),        # Latitude as ddmm.mm - dd is degrees, mm.mm is minutes
            ('ns', 'U1'),         # N or S (North or South)
            ('lon', 'f8'),        # Longitude as ddmm.mm - dd is degrees, mm.mm is minutes
            ('ew', 'U1'),         # E or W (East or West)
            ('sog', 'f4'),        # Speed over ground, knots
            ('cog', 'f4'),        # Track made good, degrees true
            ('ddmmyy', 'u4'),     # Date, ddmmyy
            ('magVar', 'f4'),     # Magnetic Variation, degrees
            ('magVarEw', 'U1')    # E or W
        ]

        if numFields >= 12:
            dtype.extend([
                # FAA mode indicator (NMEA 2.3 and later)
                ('mode', 'U1')
            ])

        if numFields >= 13:
            dtype.extend([
                # Nav Status (NMEA 4.1 and later)
                # A=autonomous, D=differential, E=Estimated, M=Manual input mode N=not valid, S=Simulator, V = Valid
                ('status', 'U1')
            ])

        # Forward compatibility / future-proofing in case RMC payloads exceeds 13 fields
        for i in range(numFields - 13):
            dtype.extend([
                ('unknown_{:02}'.format(i), 'U1')
            ])

        return np.dtype(dtype)


    def getGgaDatatype(self, numFields):
        '''Get the datatype of the GGA payload'''

        # Standard GGA payload
        dtype = [
            ('hhmmss', 'f4'),     # UTC of this position report as hhmmss.ss
            ('lat', 'f8'),        # Latitude as ddmm.mm - dd is degrees, mm.mm is minutes
            ('ns', 'U1'),         # N or S (North or South)
            ('lon', 'f8'),        # Longitude as ddmm.mm - dd is degrees, mm.mm is minutes
            ('ew', 'U1'),         # E or W (East or West)
            ('quality', 'u1'),    # GPS Quality Indicator (non null)
            ('sats', 'u1'),       # Number of satellites in use, 00 - 12
            ('hdop', 'f4'),       # Horizontal Dilution of precision (meters)
            ('alt', 'f4'),        # Antenna Altitude above/below mean-sea-level (geoid) (in meters)
            ('altUnits', 'U1'),   # Units of antenna altitude, meters
            ('geoSep', 'f4'),     # Geoidal separation, the difference between the WGS-84 earth ellipsoid and mean-sea-level
            ('geoSepu', 'U1'),    # Units of geoidal separation, meters
            ('age', 'u2'),        # Age of differential GPS data, time in seconds since last SC104 type 1 or 9 update
            ('dgpsId', 'u2')      # Differential reference station ID, 0000-1023
        ]

        # Forward compatibility / future-proofing in case GGA payloads exceeds 14 fields
        for i in range(numFields - 14):
            dtype.extend([
                ('unknown_{:02}'.format(i), 'U1')
            ])

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # TODO
        return

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['lat'] / 10000000
        self.data['lon'] = self.rawData['lon'] / 10000000

        # Convert elevation from cm to m
        self.data['ele'] = self.rawData['alt']

        # Convert SOG from knots to m/s
        self.data['sog'] = self.rawData['sog'] / 1000

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['cog']

        # Convert date/time to regular timestamp (seconds)
        vfunc = np.vectorize(getDateTime)
        timestampSecs = vfunc(  self.rawData['year'], self.rawData['month'], self.rawData['day'],
                                self.rawData['hour'], self.rawData['min'], self.rawData['sec'])

        # Convert timestamps to milliseconds
        self.data['timestamp'] = timestampSecs + np.round(self.rawData['nano'] / 1000000000, 3)

        # Copy fix and satellites
        self.data['fix'] = self.rawData['fixType']
        self.data['sat'] = self.rawData['numSV']

        # Convert PDOP from integer to decimal
        self.data['hdop'] = self.rawData['hdop'] / 100
            

def getDateTime(year, month, day, hour, minute, second):
    '''Decode date/time'''

    return datetime(year, month, day, hour, minute, second, tzinfo=timezone.utc).timestamp()

## Unit Tests

In [3]:
class TestGt31Data(unittest.TestCase):
    '''Class to test data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(gt31Reader.numRecords, 4161)

        for fieldName in gt31Reader.data:
            self.assertEqual(gt31Reader.data[fieldName].size, 4161)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(gt31Reader.data['hdop'].min(), 0.8)
        self.assertEqual(gt31Reader.data['hdop'].max(), 2.0)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(gt31Reader.data['sat'].min(), 4)
        self.assertEqual(gt31Reader.data['sat'].max(), 10)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(gt31Reader.data['timestamp'].min(), 1649672182)
        self.assertEqual(gt31Reader.data['timestamp'].max(), 1649678792)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(gt31Reader.data['lat'].min(), 50.571016)
        self.assertEqual(gt31Reader.data['lat'].max(), 50.5833319)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(gt31Reader.data['lon'].min(), -2.4620455)
        self.assertEqual(gt31Reader.data['lon'].max(), -2.4563038)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(gt31Reader.data['ele'].min(), -3.02)
        self.assertEqual(gt31Reader.data['ele'].max(), 11.93)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(gt31Reader.data['sog'].min(), 0.13)
        self.assertEqual(gt31Reader.data['sog'].max(), 16.83)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(gt31Reader.data['cog'].min(), 0.01)
        self.assertEqual(gt31Reader.data['cog'].max(), 359.92)

In [4]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.nmea')
    gt31Reader = NmeaReader(filename)

    filename = os.path.join(projdir, 'sessions', '20130801-ftech', 'GPSXX006_p1.nmea')
    ftechReader = NmeaReader(filename)

    pc1 = time.perf_counter()
    gt31Reader.load()
    ftechReader.load()
    pc2 = time.perf_counter()

    print("\nUBX files loaded in in %0.2f seconds" % (pc2 - pc1))


UBX files loaded in in 2.02 seconds


In [5]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    #unittest.main(argv=['first-arg-is-ignored'], exit=testExit)